 #  <center>SpaceShip-Titanic Tutorial </center>
 
1. In this tutorial, we are going to be **making a classification model to make predictions on the spaceship titanic dataset**. Being a beginners tutorial, all you need to have some basic knowledge in statictics and python.
 
2. In this tutorial, we will be making use of Pandas Profiling to make detailed statistical reports about the dataset, you'll see for yourself the ultimate power of **Pandas Profiling** and its use for **fast pacing the construction of Data reports**, something we (data scientists) used to do by importing a libraries and creating instances.

3. Although we will be using off the shelf classification algorithms and trainging them, we will no tbe going into detail of how the algorithms work, although **you'll be learning how to make model pipelines and creating dictionaries in model construction**.
 
3. In the end, if you like this tutorial, **it'll mean alot to me if you upvote the tutorial**, I also have some more awesome content dropping **SOON** with more indepth statistical analysis into complex datasets.
 
 # Table of Contents
    
<a id="toc"></a>
 
 - [1. Taking a Look at the Data and make some notes](#1)
 - [2. Do fast EDA using Pandas Profiling](#2)
 - [4. Data Preprocessing and Feature Engineeing](#4)
 - [5. Model Construction](#5)
 - [6. Predictions](#6)
 - [7. Submission](#7)
 




<a id="1"></a>
# Take a Look at our Data and Make some Notes

# Import Libraries

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
### Read in and Take a Look at our data
df=pd.read_csv('../input/spaceship-titanic/train.csv')

In [ ]:
df.head()

In [ ]:
##See whether we have any null values in our Dataset
df.isnull().sum()

##Well well well...... we do have a lot of null values in our dataset, hmmm..... we'll handle this later, let us look at the statistics of our dataset.... this is something i do to get an idea of the distribution of the data.

In [ ]:
df.describe()

# **My Notes**

1. People ina  group are often families bt not always, _02 in the family ID then its a group travelling
2. cabin is split up into deck number and side so we may want to split thatup
3. Room Service, FOOD coURT, VR DECK SPA AND SHOPPING MALL ARE MONETARY VALUES 
4. Machine Learning Task is to predict transported
Its a binary Classification Problem.



To Do
1. Fill in Missing Values in the columns
2. Also Balance out the dataset if imbalanced.
3.Drop high cardinality columns


If you are doing a real data sceince project, you go to the client and speak to them about the data, you ask them about the relationships present in the dayta and about the variable they are trying to predict.

<a id="2"></a>
# **Do Fast EDA using Pandas Profiling**

Pandas Profiling is an extremely useful tool for y'all data science enthusiasts.  The pandas df.describe() function is handy yet a little basic for exploratory data analysis. pandas-profiling extends pandas DataFrame with df.profile_report(), which automatically generates a standardized univariate and multivariate report for data understanding.

In [ ]:
!pip install pandas-profiling

In [ ]:
# Import the library which we just installed
from pandas_profiling import ProfileReport

In [ ]:
#Call the function from the library and enter the dataframe you want to do EDA on
profile =ProfileReport(df,'Spaceship-Titanic')

# What the Profile Report Gives you

0. Type inference: detect the types of columns in a DataFrame
1. Essentials: type, unique values, indication of missing values
2. Quantile statistics: minimum value, Q1, median, Q3, maximum, range, interquartile range
3. Descriptive statistics: mean, mode, standard deviation, sum, median absolute deviation, 4. 4. coefficient of variation, kurtosis, skewness
5. Most frequent and extreme values
6. Histograms: categorical and numerical
7. Correlations: high correlation warnings, based on different correlation metrics (Spearman, 8. Pearson, Kendall, Cramér’s V, Phik)
9. Missing values: through counts, matrix, heatmap and dendrograms
10. Duplicate rows: list of the most common duplicated rows

In [ ]:
profile.to_notebook_iframe()

Lets focus on the Cabin Column: According to the definitions of each column provided in the dataset, The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard. Lets divide this into Deck and Side Feature, for now I'll treat the deck number as not that relevant

In [ ]:
# This function will later be used in the preprocessing pipeline, right now, its just being used
# to see what we can do with the data

def split_cabin(x):
  if len(str(x).split('/'))<3:

    return ['Missing','Missing','Missing']


  else:
    return str(x).split('/')


df['TempCabin']=df['Cabin'].apply(lambda x: split_cabin(x))
df['Deck']=df['TempCabin'].apply(lambda x: x[0])
df['Side']=df['TempCabin'].apply(lambda x: x[2])

In [ ]:
##Lets take another look at our data now, augmented with 3 more columns
df.head()

In [ ]:
##Drop the Temp Cabin Feature, this is a part of the data cleaning proces.
df.drop(['TempCabin'],axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
# Now we have about 14 feature columns and one label column, further data cleaning will be done to 
# Cater to the categorical features and numerical features, operations such as dummy creation for categorical variables
#and data imputation for numerical variables.
df.info()

<a id="4"></a>
# Data Preprocessing and Feature Engineering

In [ ]:
##Create a preprocessing function to transform our dataset,
##this preprocessing pipeline does it all i.e., feature engineering and Data cleaning  
def preprocessing (df):

  #Fill missing values with missing
  df['HomePlanet'].fillna('Missing',inplace=True)
  df['CryoSleep'].fillna('Missing', inplace=True)
  
  #Cabin Preprocesing, extract Deck and S ide
  df['TempCabin']=df['Cabin'].apply(lambda x: split_cabin(x))
  df['Deck']=df['TempCabin'].apply(lambda x: x[0])
  df['Side']=df['TempCabin'].apply(lambda x: x[2])
  df.drop(['TempCabin','Cabin'], axis=1, inplace=True)
  
  #destination
  # Fill null values with 'missing'
  df['Destination'].fillna('Missing', inplace=True)
  
  #imputing the mean for age

  df['Age'].fillna(df['Age'].mean(), inplace=True)
  #VIP - drop na rows
  df['VIP'].fillna('Missing', inplace=True)
  
  #Monetary spending columns:RoomService, FoodCourt, ShoppingMall, Spa,
  #VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's
  #many luxury amenities.
  df['RoomService'].fillna(0, inplace=True)
  df['FoodCourt'].fillna(0, inplace=True)
  df['ShoppingMall'].fillna(0, inplace=True)
  df['Spa'].fillna(0, inplace=True)
  df['VRDeck'].fillna(0, inplace=True)
  #Drop name = high cardinality
  df.drop('Name', axis=1, inplace=True)
  #Drop remaining rows

  return df

# Prepare Dataset from Model

In [ ]:
df=pd.read_csv('../input/spaceship-titanic/train.csv')
abt=df.copy()
abt.info()

In [ ]:
##Call the Preprocessing function on your trainuing dataset.
preprocessing(abt)

In [ ]:
#8693 unique records
##We see that we have reduced the entries and have also dropped some columns
abt.info()

<a id="4"></a>
# Model Construction

- Feature and Target values - X, y
- One hot encode any categorical features
- Train, holdout split
- Train on a bunch of algos

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
#To check whether our dataset is balanced or not 
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
#Creating labels and features columns
X= abt.drop(['Transported','PassengerId'],axis=1)
#One hot encode
X=pd.get_dummies(X)
y=abt['Transported']

**Train-Test-Split Partitioning**

In [ ]:
#Create training and testing partitions
X_train, X_test, y_train, y_test=train_test_split(X, y , test_size=0.3, random_state=1234)

In [ ]:
sns.countplot(x='Transported', data=df)
##It looks like our dataset is pretty much balanced, so what happens is that when we are working on binary classification problems,
##we would want our dataset to be balanced with equal numbers of booleans
##For a reasobnably well performing model

In [ ]:
# Take a look at our training data
X_train.info()

In [ ]:
X_test.shape

In [ ]:

from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Learning Algorithms to Use

 **1**. Grid Search CV: GridSearchCV is a library function that is a member of sklearn's model_selection package. It helps to loop through predefined hyperparameters and fit your estimator (model) on your training set. So, in the end, you can select the best parameters from the listed hyperparameters. Learn out more in depth about Grid Search CV using the following link : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

**2**. Random Forest Classifier and Gradient Boosting Classifier which are both ensemble learning algorithms, find out more about them using this link : https://scikit-learn.org/stable/modules/ensemble.html#

In [ ]:
##Setup our Machine Learnig Pipelines
pipelines={
    'rf':make_pipeline(StandardScaler(),RandomForestClassifier(random_state=1234)),
    'gb':make_pipeline(StandardScaler(),GradientBoostingClassifier(random_state=1234))

}

##The standard scaler essentially does x-mu divided by sigma, it scales the dataset to center around zero mean

In [ ]:
grid={
    'rf':{
        'randomforestclassifier__n_estimators':[100,200,300]
    },
    'gb':{
        'gradientboostingclassifier__n_estimators':[100,200,300]
    }
}

In [ ]:
RandomForestClassifier().get_params()

# Tuning Loop
Were tuning, we tune using a gridsearch cv its going to scan across our tuning grid like our hyperparameters and also perform crossvalidation

In [ ]:
pipelines.items()

In [ ]:
#Create a blank dictionary to hold models
fit_models={}
#Loop through al the algos
for algo, pipeline in pipelines.items():
  print(f'training the {algo} model.')
  #Create new grid search cv class
  model=GridSearchCV(pipeline, grid[algo], n_jobs=-1, cv=10)
  # Train the model
  model.fit(X_train, y_train)
  #Store the results inside of our dictionary
  fit_models[algo]=model

# Evaluate model on the Test Partition
- Grab the testing data

In [ ]:
fit_models

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
#Evaluate the performance of the models

for algo, model in fit_models.items():
  y_hat=model.predict(X_test)
  print(accuracy_score(y_test, y_hat))
  accuracy=accuracy_score(y_test,y_hat)
  recall=recall_score(y_test,y_hat)
  precision=precision_score(y_test,y_hat)
  print(f'Metrics for {algo}: accuracy -{accuracy}, recall-{recall}, precision-{precision}')


In [ ]:
y_hat

# Save Best Model

In [ ]:
import pickle
with open('gradientboosted.pkl','wb') as f:
  pickle.dump(fit_models['gb'],f)
with open('gradientboosted.pkl','rb') as f:
  reloaded_model=pickle.load(f)

<a id="6"></a>
# Predict our Test Data

In [ ]:
#Read in the test CSV dataset
test_df=pd.read_csv('../input/spaceship-titanic/test.csv')
#Create a Deep copy of the test dataset
abt_test=test_df.copy()
#Run the deep copy through the preprocessing pipeline
preprocessing(abt_test)
#One hot encode categorical data
abt_test=pd.get_dummies(abt_test.drop('PassengerId', axis=1))

In [ ]:
abt_test.head()

In [ ]:
yhat_test=fit_models['gb'].predict(abt_test)

In [ ]:
submission=pd.DataFrame([test_df['PassengerId'],yhat_test]).T
submission.columns=['PassengerId','Transported']

In [ ]:
submission.head()

<a id="7"></a>
# Submitting Our Work

In [ ]:
# Convert submission dataframe to a csv file (A comma separated Value File)
submission.to_csv('submission.csv', index=False)